## Домашнее задание

__Задание 1. Генератор случайных матриц.__

Реализовать генератор матриц, который должен поддерживать функции:
* Генерация абсолютно случайной матрицы $n\times m$
* Генерация случайной диагональной матрицы $n\times n$
* Генерация случайной верхнетреугольной матрицы
* Генерация случайной нижнетреугольной матрицы
* Генерация симметричной матрицы
* Генерация возмущения матрицы $n\times m$, каждый элемент которой не превосходит по модулю заданный $\varepsilon$. Оценить величину нормы матрицы возмущений в зависимости от параметра $\varepsilon$ (оценить верхную границу).

Оценить численно вероятность того, что созданная матрица будет вырожденной для какого-либо случая выше.

In [2]:
import numpy as np
from scipy.linalg import eigvals

In [3]:
def matrix_generate(rows, columns, type_ = "full", eps = 0):
    """
    matrix_generate(rows, columns, type_ = "full")

    Создаёт случайную матрицу выбранного типа.

    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".

    Parameters
    ----------

    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "upper_triangular", "symmetric" и т.д.
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц.

    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.

    Notes
    -----
    Поддерживаемые типы матриц:
        "full","diagonal","upper_triangular","lower_triangular",
        "symmetric", "perturbation"
    """

    A = None
    if type_ == "full":
        A = np.random.random(size=(rows, columns))
    elif type_ == "diagonal":
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.diag(np.random.random(rows))
    elif type_ == "upper_triangular":
        A = np.triu(np.random.random(size=(rows, columns)))
    elif type_ == "lower_triangular":
        A = np.tril(np.random.random(size=(rows, columns)))
    elif type_ == "symmetric":
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.random.random(size=(rows, columns))
            A = np.triu(A) + np.triu(A, 1).T
    elif type_ == "perturbation":
        A = np.random.uniform(low=-eps, high=eps, size=(rows, columns))
        frob_norm_bound = eps * np.sqrt(rows * columns)
        spectral_norm_bound = eps * min(rows, columns)

        print(f"Верхняя оценка нормы Фробениуса: {frob_norm_bound:.6f}")
        print(f"Верхняя оценка спектральной нормы: {spectral_norm_bound:.6f}")
    else:
        return f"Error with type {type_}"
    return A

Верхняя граница нормы матрицы возмущения:

Максимальная векторная норма - eps*columns

Манхэттенская норма - eps*rows

Евклидова норма - $<= \sqrt{||A||_1 ||A||_\infty} = eps*\sqrt{rows*columns}$

Норма Фробениуса - $eps*\sqrt{rows*columns}$

Для всех вышеперечисленных матриц, кроме матрицы возмущения с параметром eps=0, вероятность вырождения равна нулю.

__Задание 2. Вычисление матричных норм и числа обусловленности.__

Реализовать вычисление трех основных норм векторов (L1, L2 и максимальную) и подчиненных им матричных норм. Реализовать вычисление числа обусловленности.

Примечание: для вычисления собственных значений можно использовать linalg.eigvals из модуля scipy.

In [1]:
def vector_norm(x, type_='l2'):
    """
    Нормы вектора
    'l1' - сумма абсолютных значений
    'l2' - евклидова норма
    'max' - максимальное по модулю значение
    """
    if type_ == 'l1':
        return np.sum(np.abs(x))
    elif type_ == 'l2':
        return np.sqrt(np.sum(x**2))
    elif type_ == 'max':
        return np.max(np.abs(x))
    else:
        raise f"Error with type {type_}"

def matrix_norm(A, type_='l2'):
    """
    Матричные нормы
    'l1' - максимальная сумма по столбцам
    'l2' - максимальное сингулярное число
    'max' - максимальная сумма по строкам
    """
    if type_ == 'l1':
        return np.max(np.sum(np.abs(A), axis=0))
    elif type_ == 'l2':
        eigenvalues = eigvals(A.T @ A)
        max_eigenvalue = np.max(np.real(eigenvalues))
        return np.sqrt(max_eigenvalue)
    elif type_ == 'max':
        return np.max(np.sum(np.abs(A), axis=1))
    else:
        raise f"Error with type {type_}"

def condition_number(A, type_='l2'):
    """
    Допустимые type_ в matrix_norm()
    """
    try:
        norm_A = matrix_norm(A, type_)
        norm_A_inv = matrix_norm(np.linalg.inv(A), type_)
        return norm_A * norm_A_inv
    except np.linalg.LinAlgError:
        return np.inf

__Задание 3. Эквивалентность первых двух норм.__

Найдите константы $C_1$  и  $C_2$ такие, что

$\ C_1||\mathbf{x}||_2\leq||\mathbf{x}||_1\leq C_2||\mathbf{x}||_2$


Указание: в качестве подсказки можно использовать визуализацию норм из документа с теорией.

Из визуализации норм видно, что $C_1=1$.

Для второго неравенства воспользуемся неравенством Коши Буянковского.
$$
||x||_1 = \sum |x_i| \leq \sqrt{\sum x_i^2} \sqrt{\sum 1^2} = ||x||_2 \sqrt{n}
$$
Следовательно $C_2 = \sqrt{n}$

__Задание 4. Евклидова и бесконечная норма.__

 Пусть x — вектор размерности m, а A — матрица m×n. Докажите следующие неравенства и приведите
примеры таких x и A, при которых неравенства обращаются в равенства:

- $\|x\|_2 \leq \sqrt{m} \cdot \|x\|_{\infty}$
- $\|A\|_{\infty} \leq \sqrt{n} \cdot \|A\|_2$

Первое неравенство превращается в равенство, если элементы вектора одинаковы.
$$
\sqrt{\sum x_i^2} \leq \sqrt{\sum (max(|x_i|))^2 } = \sqrt{m} max(|x_i|)
$$



Во втором неравенстве рассмотрим норму $||A||_2$.
По одному из определений:
$$
||A||_2 = max||Ax||_2, ||x||_2 = 1.
$$
Так как мы можем брать любой вектор $x$, возьмем его так, чтобы $Ax$ было строкой матрицы $A$ (обозначим $a_i$).

Тогда $||A||_2 \geq max||a_i||_2$.

Также для нормы $||A||_\infty= max||a_i||_1$, имеем теорему $||a_i||_1 \leq\sqrt{n}||a||_2$.

Следовательно,
$$\sqrt{n}||A||_2 \geq \sqrt{n} max||a_i||_2 \geq max ||a_i||_1 = ||A||_\infty $$
Пример, когда оба неравенства превращаются в равенство - это максимальная строка состоит из одинаковых чисел
и и выполняется равенство в первой части. Например, матрица с единственной строкой, состоящей из единиц.

__Задание 5.  Норма Фробениуса.__

Докажите, что для любой унитарной матрицы U (и для произвольной матрицы A) имеет место равенство

 $\| U A \|_F = \| AU \|_F = \| A \|_F$ ,

 где $\| \cdot \|_F$ — норма Фробениуса.

Указание.
Задачу можно решить без вычислений, если использовать геометрический смысл нормы Фробениуса и геометрические свойства умножения на унитарную матрицу (что при умножении на неё сохраняется).

Геометрически нормы Фробениуса это l2 норма вектора образованного из всех столбцов матрицы. Если $a_i$ столбцы матрицы, то $||A||_F = \sqrt{\sum||a_i||_2^2}$. Известно, что унитарное преобразование вектора сохраняет евклидову норму, следовательно, нормы столбцов $a_i$ не изменяются, а значит норма Фробениуса не поменяется. Для другой части равенства, рассуждения такие же, но для строк.

__Задание 6*.  Тензорная свёртка.__

Рассмотрим функцию, отображающую шесть тензоров на один тензор: $Z\left(\lambda^{(1)}, \lambda^{(2)}, \lambda^{(3)}, \Gamma^{(1)}, \Gamma^{(2)}, U\right)$ :


$$
Z_{a h i j}=\sum_{b c d e f g} \lambda^{(1)}{ }_{a b} \Gamma_{c b d}^{(1)} \lambda^{(2)}{ }_{d e} \Gamma_{f e g}^{(2)} \lambda_{g h}^{(3)} U_{i j c f}
$$

редположив, что все индексы пробегают значения от 1 до χ, проведите эксперимент и сравните скорость
различных реализаций функции Z. Исследуйте значения χ в диапазоне 3–50.


- В файле convolution.ipynb вы можете найти релизацию глупого способа вычисления этой свертки, который требует $\chi^4 \times \chi^6=\chi^{10}$ операций. На самом деле это можно вычислить гораздо быстрее!
- С помощью функции numpy.einsum (нужно использовать аргумент optimize), можно добиться намного большей производительности. Чтобы понять, что происходит под капотом, воспользуйтесь функцией numpy.einsum_path. Какое минимальное количество операций требуется для вычисления $Z$ ?
- Посмотрев на вывод функции numpy.einsum_path, peализуйте алгоритм для вычисления $Z$, который столь же эффективен, как numpy.einsum, но использует более элементарные numpy.dot и numpy.tensor_dot.
